https://github.com/Kaggle/kaggle-api

https://www.kaggle.com/c/airbnb-pricing/data

In [ ]:
%%bash

rm -rf data
kaggle competitions download -c airbnb-pricing -p data/


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('data/train.csv.gz', compression='gzip')
test = pd.read_csv('data/test.csv.gz', compression='gzip')

subm = pd.read_csv('data/sample_submission.csv.gz', compression='gzip')

In [ ]:
# Удаляем квартиры с нулевой ценой

train = train.drop(train[train.price == 0].index)

### метрика MAPE

In [ ]:
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred) / (y_true)).replace([-np.inf, np.inf], np.nan).dropna())*100

### константный бейзлайн

In [ ]:
# Медиана

In [ ]:
np.median(train.price)

In [ ]:
# Качество на трейне с константой 
MAPE(train.price, 80)

In [ ]:
# Отправим константное решение 
subm['price'] = 80
#subm.to_csv('./submissions/const.csv', index=False)

MAPE(y_test[y_test.is_public == 1].price, [80])

In [ ]:
# Взвешенная медиана 

In [ ]:
# Вектор нормализованных весов для y_train

inv_y = 1 / train.price[train.price != 0].values

w = inv_y / sum(inv_y)

In [ ]:
#сортируем веса
idxs = np.argsort(w)
sorted_w = w[idxs]

# Вычисляем накопительную сумму
sorted_w_cumsum = np.cumsum(sorted_w)

# находим индекс, для которого накопительная сумма становится больше 0.5
idx = np.where(sorted_w_cumsum > 0.5)[0][0]

# находим индекс искомого значения
pos = idxs[idx]
train.price[train.price != 0].values[pos]

In [ ]:
# качество на трейне с константой 
MAPE(train.price, 45)

In [ ]:
# отправим константное решение 
subm['price'] = 45
subm.to_csv('./submissions/const.csv', index=False)

In [ ]:
# Если Вы счастливый обладатель невинды отправлять решения можно вот так 

In [ ]:
%%bash

kaggle competitions submit -f ./submissions/const.csv -m constant_submission -c airbnb-pricing

In [ ]:
%%bash

kaggle competitions airbnb-pricing

### кросс валидация

In [ ]:
# Объединим трейн и тест

test['price'] = -1

data = pd.concat([train, test], 0)
data.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# Эта функция определяет что нам сейчас нужно на некотором наборе данных. Кросс валидация, или предсказания для теста.

def cv_mode(CV):
    
    if CV:
    
        cv = KFold(n_splits=5, shuffle=False, random_state=42).split(data.iloc[:train.shape[0]])
    
    else:
        i_tr = data.iloc[:train.shape[0]].index
        i_tst = data.iloc[train.shape[0]:].index
        cv = (i_tr, i_tst)
    
    return cv

In [ ]:
# Получить предсказания, кросс-валидация, файл с решением

def make_prediction(model, X, y, i_tr, i_tst):
    X_train, X_test = X.iloc[i_tr, :], X.iloc[i_tst, :]
    y_train, y_test = y.iloc[i_tr], y.iloc[i_tst]
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    return prediction, MAPE(y_test, prediction)
        
    
def cross_val(model, X, y, CV=True):
    cv = cv_mode(CV)
    scores = []
    for i_tr, i_tst in cv:
        _, score = make_prediction(model, X, y, i_tr, i_tst)
        scores.append(score)
    return np.mean(scores), np.std(scores)


def make_subm(model, X, y, filename, CV=False):
    cv = cv_mode(CV)
    (i_tr, i_tst) = cv
    prediction, _ = make_prediction(model, X, y, i_tr, i_tst)
    subm['price'] = prediction
    subm.to_csv(filename, index=False)
    pass



### baselines

In [ ]:
# Возьмем числовые признаки
num_cols = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'square_feet',
            'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
            'minimum_nights', 'latitude', 'longitude']

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor

In [ ]:
X = data[num_cols].fillna(0)
y = data.price

In [ ]:
# Почему скор хуже константы?
model = Ridge()
cross_val(model, X, y)

In [ ]:
# Еще немного про MAPE

In [ ]:
model = LGBMRegressor()
cross_val(model, X, y)

In [ ]:
# добавить веса

model = LGBMRegressor()
def make_prediction( model, X, y, i_tr, i_tst):
    X_train, X_test = X.iloc[i_tr, :], X.iloc[i_tst, :]
    y_train, y_test = y.iloc[i_tr], y.iloc[i_tst]
    model.fit(X_train, y_train, sample_weight=1/y_train)
    prediction = model.predict(X_test)
    return prediction, MAPE(y_test, prediction)
           
cross_val(model, X, y)

In [ ]:
# логарифмируем таргет

model = LGBMRegressor()

def make_prediction( model, X, y, i_tr, i_tst):
    X_train, X_test = X.iloc[i_tr, :], X.iloc[i_tst, :]
    y_train, y_test = np.log1p(y.iloc[i_tr]), y.iloc[i_tst]
    model.fit(X_train, y_train)
    prediction = np.expm1(model.predict(X_test))
    return prediction, MAPE(y_test, prediction)
        
    
cross_val(model, X, y)

In [ ]:
# Проверим насколько наша валидация согласуется с лидербордом

In [ ]:
scores = []
for i, model in enumerate([Ridge(random_state=42), 
                          RandomForestRegressor(random_state=42, n_jobs=-1), 
                          KNeighborsRegressor(n_jobs=-1), LGBMRegressor()]):
    
    score, _ = cross_val(model, X, y, CV=True)
    scores.append(score)
    make_subm(model, X, y, cv, 'subm'+str(i)+'.csv')

In [ ]:
ldb_scores=[43.845, 31.545, 41.248, 28.129]

In [ ]:
plt.scatter(scores, ldb_scores)